<img src="../artworks/matchzoo-logo.png" alt="logo" style="width:600px;float: center"/>

# MatchZoo Quick Start

In [66]:
import matchzoo as mz
print(mz.__version__)
import pandas as pd

2.2.0


## Define Task

There are two types of tasks available in MatchZoo. `mz.tasks.Ranking` and `mz.tasks.Classification`. We will use a ranking task for this demo.

In [2]:
#task = mz.tasks.Ranking()
#print(task)

## Prepare Data

`DataPack` is a MatchZoo native data structure that most MatchZoo data handling processes build upon. A `DataPack` is consists of three `pandas.DataFrame`:

It is also possible to convert a `DataPack` into a single `pandas.DataFrame` that holds all information.

In [67]:
import pickle
with open('/home/deepti-saravanan/Desktop/Ranking_Attention/matchzoo/datasets/trainquery.pkl','rb') as f:
    querydata = pickle.load(f)
with open('/home/deepti-saravanan/Desktop/Ranking_Attention/matchzoo/datasets/finaltrain.pkl','rb') as f:
    ansdata = pickle.load(f)

In [68]:
with open('/home/deepti-saravanan/Desktop/QnA_deploy/tfidf_regs.pkl','rb') as f:
    tfidf = pickle.load(f)

In [4]:
vocab = list(tfidf.keys())

In [37]:
idl = []
idr = []
text_left = []
text_right = []
text_freq = []
dot_right = []
dot_left = []
label = []
text_freq0 = []
t=0
while(t<len(querydata)):
    f = []
    for i in querydata[t].split():
        if(i in vocab):
            f.append(int(tfidf[i]))
        else:
            f.append(0)
    ans = ansdata[t]
    k=0
    while(k<len(ans)):
        if(type(ans[k]) == str):
            idval1 = t+1
            idval2 = k+1
            idl.append('Q'+ str(idval1))
            idr.append('A'+str(idval1)+'-'+str(idval2))
            text_left.append(querydata[t])
            text_right.append(ans[k])
            dot_right.append([1]*30)
            dot_left.append([1])
            label.append(float(k))
            if(len(f) < 30):
                padding = len(f)
                while(padding < 30):
                    f.append(0)
                    padding+=1
            elif(len(f) > 30):
                f0 = f[0:30]
                f = f0
            else:
                lol=0
            text_freq.append(f)
            #text_freq.append(1)
        
        k+=1
    t+=1

In [38]:
with open('text_freq.pkl','wb') as f:
    pickle.dump(text_freq,f)

In [39]:
data = pd.DataFrame({
    'id_left' : idl,
    'text_left': text_left,
    'id_right' : idr,
    'text_right': text_right,
    'text_freq' : text_freq,
    'dot_right' : dot_right,
    'dot_left' : dot_left,
    'label' : label
})
my_pack = mz.pack(data)
my_pack.frame().head()

,id_left,text_left,id_right,text_right,text_freq,dot_right,dot_left,label
0,Q1,Who is restrained or prohibited from accessing...,A1-1,"(2), (2A) and (3) of section 11 and section 11...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 0, 0, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[1],1.0
1,Q1,Who is restrained or prohibited from accessing...,A1-2,(2) Dealing in securities shall be deemed to ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 0, 0, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[1],2.0
2,Q1,Who is restrained or prohibited from accessing...,A1-3,"7. The Investigating Authority may, after obta...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 0, 0, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[1],3.0
3,Q1,Who is restrained or prohibited from accessing...,A1-4,3. Prohibition of certain dealings in securiti...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 0, 0, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[1],4.0
4,Q1,Who is restrained or prohibited from accessing...,A1-5,"5. Where the Board, the Chairman, the m...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 0, 0, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[1],5.0


In [40]:
frame = my_pack.frame
my_pack.relation['label'] = my_pack.relation['label'] + 1

In [41]:
num_train = int(len(my_pack) * 0.8)
my_pack.shuffle(inplace=True)
train_pack = my_pack[:num_train]
valid_pack = my_pack[num_train:]

However, using such `pandas.DataFrame` consumes much more memory if there are many duplicates in the texts, and that is the exact reason why we use `DataPack`. For more details about data handling, consult `matchzoo/tutorials/data_handling.ipynb`.

## Preprocessing

MatchZoo preprocessors are used to convert a raw `DataPack` into a `DataPack` that ready to be fed into a model. 

In [42]:
preprocessor = mz.preprocessors.BasicPreprocessor()

There are two steps to use a preprocessor. First, `fit`. Then, `transform`. `fit` will only changes the preprocessor's inner state but not the input `DataPack`.

In [43]:
preprocessor.fit(train_pack, verbose=0)

`fit` will gather useful information into its `context`, which will be used later in a `transform` or used to set hyper-parameters of a model.

In [44]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f0e13063ac8>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f0d6a0c1668>,
 'vocab_size': 6937,
 'embedding_input_dim': 6937,
 'input_shapes': [(30,), (30,)]}

Once `fit`, the preprocessor has enough information to `transform`.  `transform` will not change the preprocessor's inner state and the input `DataPack`, but return a transformed `DataPack`.

In [45]:
# Commenting out for now

train_processed = preprocessor.transform(train_pack, verbose=0)
test_processed = preprocessor.transform(valid_pack, verbose=0)

In [15]:
#train_processed0 = train_processed
#test_processed0 = test_processed

In [46]:
%store train_processed

Stored 'train_processed' (DataPack)


In [47]:
%store test_processed

Stored 'test_processed' (DataPack)


In [48]:
train_processed.frame().head()

,id_left,text_left,length_left,id_right,text_right,length_right,text_freq,dot_right,dot_left,label
0,Q4626,"[2801, 2148, 4614, 6642, 1902, 813, 1277, 2879...",27,A4626-16,"[5670, 4155, 2, 5853, 5261, 4984, 2297, 111, 1...",30,"[0, 2, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[1],17.0
1,Q3786,"[721, 1604, 5670, 4155, 2046, 6763, 4578, 5670...",21,A3786-6,"[1232, 913, 312, 3810, 5670, 5741, 1232, 913, ...",30,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[1],7.0
2,Q7403,"[2801, 4614, 5670, 4155, 1232, 3242, 4462, 638...",9,A7403-18,"[4155, 2, 2379, 1232, 653, 3046, 5953, 4677, 1...",30,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[1],19.0
3,Q652,"[5670, 6255, 6684, 2459, 6171, 3155, 813, 4383...",22,A652-17,"[6171, 1948, 6334, 5465, 5853, 2459, 3176, 171...",30,"[0, 0, 0, 54, 0, 0, 0, 0, 0, 50, 0, 0, 0, 0, 0...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[1],18.0
4,Q2997,"[4720, 1604, 5670, 65, 878, 1232, 391, 4963, 3...",13,A2997-42,"[5830, 5737, 5853, 6635, 173, 4401, 5853, 3046...",30,"[0, 0, 0, 0, 31, 0, 0, 79, 0, 0, 0, 0, 0, 0, 0...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[1],43.0


As we can see, `text_left` is already in sequence form that nerual networks love.

Just to make sure we have the correct sequence:

In [49]:
vocab_unit = preprocessor.context['vocab_unit']
print('Orig Text:', train_processed.left.loc['Q1']['text_left'])
sequence = train_processed.left.loc['Q1']['text_left']
print('Transformed Indices:', sequence)
print('Transformed Indices Meaning:',
      '_'.join([vocab_unit.state['index_term'][i] for i in sequence]))

Orig Text: [721, 1604, 4598, 3810, 5250, 449, 6153, 5670, 1467, 4115, 3810, 4488, 5407, 1467, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Transformed Indices: [721, 1604, 4598, 3810, 5250, 449, 6153, 5670, 1467, 4115, 3810, 4488, 5407, 1467, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Transformed Indices Meaning: who_is_restrained_or_prohibited_from_accessing_the_securities_market_or_dealing_in_securities__<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>


For more details about preprocessing, consult `matchzoo/tutorials/data_handling.ipynb`.

## Build Model

MatchZoo provides many built-in text matching models.

Let's use `mz.models.DenseBaseline` for our demo.

In [50]:
model = mz.models.ANMM()

The model is initialized with a hyper parameter table, in which values are partially filled. To view parameters and their values, use `print`.

`to_frame` gives you more informartion in addition to just names and values.

To set a hyper-parameter:

In [51]:
model.params['task'] = mz.tasks.Ranking()
model.params['task'].metrics = ['acc', 'mse', 'mae', 'ce','average_precision', 'precision', 'dcg', 'ndcg','mean_reciprocal_rank', 'mean_average_precision', 'mrr','map', 'MAP',mz.metrics.AveragePrecision(threshold=1),mz.metrics.Precision(k=2, threshold=2),mz.metrics.DiscountedCumulativeGain(k=2),mz.metrics.NormalizedDiscountedCumulativeGain(k=3, threshold=-1),mz.metrics.MeanReciprocalRank(threshold=2),mz.metrics.MeanAveragePrecision(threshold=3)]

In [51]:
'''
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)
'''

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [22]:
'''
num_neg = 4
task = mz.tasks.Ranking()
task.metrics = ['map', 'ndcg']
#task.output_dtype
#task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss(num_neg=num_neg))
model.params['task'] = task
#model.params['mlp_num_units'] = 3
'''

"\nnum_neg = 4\ntask = mz.tasks.Ranking()\ntask.metrics = ['map', 'ndcg']\n#task.output_dtype\n#task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss(num_neg=num_neg))\nmodel.params['task'] = task\n#model.params['mlp_num_units'] = 3\n"

In [52]:
model.guess_and_fill_missing_params(verbose=0)
print(model.params)

model_class                   <class 'matchzoo.models.anmm.ANMM'>
input_shapes                  [(30,), (30,)]
task                          Ranking Task
optimizer                     adam
with_embedding                True
embedding_input_dim           300
embedding_output_dim          300
embedding_trainable           True
dropout_rate                  0.1
num_layers                    2
hidden_sizes                  [30, 30]


Notice that we are still missing `input_shapes`, and that information is store in the preprocessor.

We may use `update` to load a preprocessor's context into a model's hyper-parameter table.

In [53]:
model.params.update(preprocessor.context)

Now we have a completed hyper-parameter table.

In [54]:
model.params.completed()

True

With all parameters filled in, we can now build and compile the model.

In [55]:
model.build()
model.compile()

MatchZoo models are wrapped over keras models, and the `backend` property of a model gives you the actual keras model built.

In [56]:
model.backend.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_right (InputLayer)         [(None, 30)]         0                                            
__________________________________________________________________________________________________
text_freq (InputLayer)          [(None, 30)]         0                                            
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 30)           0           text_right[0][0]                 
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 30)           0           text_freq[0][0]                  
____________________________________________________________________________________________

For more details about models, consult `matchzoo/tutorials/models.ipynb`.

## Train, Evaluate, Predict

A `DataPack` can `unpack` itself into data that can be directly used to train a MatchZoo model.

In [57]:
x, y = train_processed.unpack()
test_x, test_y = test_processed.unpack()

In [58]:
import numpy as np
ids = test_x['id_left']
idnew = []
for i in ids:
    i = i.replace('Q','')
    idnew.append(i)
    
t=0
finlist = []
while(t<len(idnew)):
    sums = 0
    k = test_x['text_left'][t]
    for i in k:
        sums = sums + i
    avgs = int(sums/len(k))
    finlist.append([avgs])
    t+=1

test_x['text_left'] = np.array(finlist)   

In [59]:
ids = x['id_left']
idnew = []
for i in ids:
    i = i.replace('Q','')
    idnew.append(i)
    
t=0
finlist = []
while(t<len(idnew)):
    sums = 0
    k = x['text_left'][t]
    for i in k:
        sums = sums + i
    avgs = int(sums/len(k))
    finlist.append([avgs])
    t+=1

x['text_left'] = np.array(finlist)   

In [60]:
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=test_x, y=test_y, batch_size=len(test_x))

In [ ]:
#glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)

In [61]:
history = model.fit(x, y, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
1168/1168 [==============================] - 5s 4ms/step - loss: 825.6197
Validation: binary_accuracy: 0.0 - mse: 741.1356811523438 - mae: 22.90125846862793 - binary_crossentropy: -359.378173828125 - average_precision(0.0): 1.0 - precision@1(0.0): 1.0 - discounted_cumulative_gain@1(0.0): 83394097430627.56 - normalized_discounted_cumulative_gain@1(0.0): 0.3529405352942191 - mean_reciprocal_rank(0.0): 1.0 - mean_average_precision(0.0): 1.0 - average_precision(1): 1.0 - precision@2(2): 0.8017591339648173 - discounted_cumulative_gain@2(0.0): 132424274323746.2 - normalized_discounted_cumulative_gain@3(-1): 0.4662995140052866 - mean_reciprocal_rank(2): 0.8486371544558283 - mean_average_precision(3): 0.7927522529304661
Epoch 2/30
1168/1168 [==============================] - 4s 4ms/step - loss: 716.7698
Validation: binary_accuracy: 0.0 - mse: 644.694580078125 - mae: 20.87962532043457 - binary_crossentropy: -359.378173828125 - average_precision(0.0): 1.0 - precision@1(0.0): 1.0 - dis

In [62]:
evals = model.evaluate(x=test_x, y=test_y)

In [63]:
evals

{'binary_accuracy': 0.0,
 'mse': 216.66637,
 'mae': 12.755432,
 'binary_crossentropy': -359.37817,
 average_precision(0.0): 1.0,
 precision@1(0.0): 1.0,
 discounted_cumulative_gain@1(0.0): 83394097430627.56,
 normalized_discounted_cumulative_gain@1(0.0): 0.3529405352942191,
 mean_reciprocal_rank(0.0): 1.0,
 mean_average_precision(0.0): 1.0,
 average_precision(1): 1.0,
 precision@2(2): 0.8017591339648173,
 discounted_cumulative_gain@2(0.0): 132424274323746.2,
 normalized_discounted_cumulative_gain@3(-1): 0.4662995140052866,
 mean_reciprocal_rank(2): 0.8486371544558283,
 mean_average_precision(3): 0.7927522529304661}

## Save and Load the Model

In [65]:
model.save('my-model')
loaded_model = mz.load_model('my-model')